In [1]:
import praw
import sys
#for time generator
from datetime import timedelta, date, datetime
from datetime import time as time_
import time
import csv
import json
from pprint import pprint
import os





def main():
    """
    
    """
    pass


def get_reddit_submission_data(from_time_interval):
    '''
    using praw search for all submissions created within this time period.
    returns a list of submissions(each likely being a dictionary)
    '''
    pass



def get_reddit_connection():
    
    try:
        from reddit_config import client_id,password,user_agent,username
        print('reddit oath credentials successfully loaded')
    except Exception as e:
        print('ERROR-0')
        print(str(e))
        
        
    reddit = praw.Reddit(client_id=client_id,
                        client_secret=password,
                        user_agent=user_agent,
                        username=username)
    return reddit
    





def insert_into_database(reddit_submission_raw_data):
    '''
    this function will update the database and append this new row to it
    '''
    pass

def get_db():
    """
    gets db, creates one if it does not exist
    """
    pass





def to_date_object(date_string):
    '''
    transforms a string from 'month/day/year' and returns equivalent date(month,day,year) object
    '''
    assert len(date_string) == 10
    date_comp = date_string.split('/')
    year = int(date_comp[2])
    month= int(date_comp[0])
    day = int(date_comp[1])
    
    return date(year,month,day)
    
    
    

def daterange(start_date, end_date):
    '''
    creates a generator that lets you iterate through a date range, one day at a time. Does not include the
    last date day just like range function.
    
    e.g:
    for single_date in daterange(start_date, end_date):
        print(single_date.strftime("%Y-%m-%d"))
    '''
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)





def save_to_csv(submission_data):
    """
    saves list of submission data to a local csv file
    """
    import datetime
    import csv
    
    time_now = str(datetime.datetime.now())[:-7]
    time_interval = submission_data[0] + '-' + submission_data[-1]
    
    
    with open("{}-{}.csv".format(time_interval,time_now, "w")) as output:
        w = csv.DictWriter(output,submission_data[0].keys())
        for row_dict in submission_data:
              w.writeheader()
              w.writerow(row_dict)
        
    
    pass
    



    
    
    
    
    
    
    
    
    
    
def collect_logging():
    """
    logs any errors
    """
    pass



#if __name__ == '__main__':
#    sys.exit(main())

In [2]:
#loads list of all nba subreddits to parse from 'nba_reddit_subnames.csv'

subreddits_list = []
with open('nba_reddit_subnames.csv',newline='') as f:
    r = csv.reader(f)
    for row in r:
        subreddits_list.append([row[0],row[1]])

In [25]:
###contains NBA season start and end dates for each seasons offseason, regular season and playoffs
###the end date is incremented by one day

season_dates = {'2013': {'offseason' : ('06/21/2013','11/28/2013'),'reg_season':('11/29/2013','04/16/2014'),'playoffs':('04/19/2014','06/15/2014')}
, 
'2014': {'offseason' : ('06/21/2014','11/28/2014'),'reg_season':('10/28/2014','04/15/2015'),'playoffs':('04/18/2015','06/16/2015')}
,
'2015': {'offseason' : ('06/17/2015','10/26/2015'),'reg_season':('10/27/2015','04/13/2016'),'playoffs':('04/16/2016','06/19/2016')} 
,
'2016' : {'offseason' : ('06/20/2016','10/24/2016'),'reg_season':('10/25/2016','04/12/2017'),'playoffs':('04/15/2017','06/12/2017')}
               }

In [ ]:


'''
subreddits_list = list of 2 item lists that contains shorthand city name /r/<subname>. ['LAL','lakers']

date_range = tuple of date objects

def reddit_Submissions_toJSON(subreddits_list,date_range):
    ([list of ['LAL','lakers']],(start_dateobject,end_dateobject)) -> returns JSON data for all submissions in this period
    
    
reddit_Submissions_toJSON(subreddits_list,season_dates['2014]['playoffs])


'''

#load oauth credentials and create reddit instance
from reddit_config import client_id,password,user_agent,username
reddit = praw.Reddit(client_id=client_id,
                        client_secret=password,
                        user_agent=user_agent,
                        username=username)

#start_date = to_date_object(date_range[0])
#end_date = to_date_object(date_range[1])
#start day =  first day of playoffs
#end date = last day of playoffs
start_date = to_date_object(season_dates['2014']['playoffs'][0])
end_date = to_date_object(season_dates['2014']['playoffs'][1])
type_season = 'playoffs'
nba_yr = '2014'


program_start_time = time.time()


json_totalseason = {'data_generated_date':datetime.now().strftime("%a, %d %b %Y %H:%M:%S -0800"),
                    'period_data':[],'runtime':''}
#for each day during the playoffs
for single_day in daterange(start_date,end_date):
    day_start = time.mktime(datetime.combine(single_day,time_(0,0,0)).timetuple())
    day_end = time.mktime(datetime.combine(single_day,time_(23,59,59)).timetuple())
    
    #this dicts holds data for the total day. Each day is stored as a list of dicts under 'data' key
    #date is date for the day of data, day_count is total # of submissions across all nba subreddits
    #errors logs any days for any subreddits where the data extraction process failed
    json_totalday = {'date':'','day_count':0,'data':[],'errors':[]}
    #counter for total submissions per day
    total_subs = 0
    for team in subreddits_list:
        #initializes our error catcher for the day
        json_error_catcher = []
        #get the /r/subredditname and pass it to praw reddit object
        subreddit = reddit.subreddit(team[1])
        
        #if this try fails that means data extraction failed for a particular day, so log the day and subreddit
        try:
            #for all submissions for 1 subreddit from start of day to end
            for submission in subreddit.submissions(day_start,day_end):
                #json_row holds our row data
                json_row = {'title':None,
                            'subreddit':None,
                            'author':None,
                            'create_date':None,
                            'submission_id':None,
                            'num_comments':None}
                
                #increment our total sub counter
                total_subs+=1
                
                #get all the data values wanted
                json_row['title']=submission.title
                json_row['subreddit']=team[0]
                json_row['author'] = submission.author.name
                json_row['create_date'] = submission.created_utc
                json_row['submission_id'] = submission.id
                json_row['num_comments'] = submission.num_comments
                
                #appends the json data row to list in totalday data. If extraction failed the values will be None    
                json_totalday['data'].append(json_row)

                
        except:
            #if extraction fails, create a list of 2 items, the string date and string name for subreddit
            #and append it to json_error_catcher list
            json_error = {'error':True,'team':team[0],'day':single_day.strftime('%Y-%m-%d')}
            json_totalday['data'].append(json_error)
            json_error_catcher.append([single_day.strftime('%Y-%m-%d'),team[0]])
            
            
        
    #records the date
    json_totalday['date'] = single_day.strftime('%Y-%m-%d')
    #records total subs for day
    json_totalday['day_count'] = total_subs
    
    #if there was an error then append the errors to the 'errors' list in json_totalday for later inspection
    #if no errors then json_totalday['errors] will be an empty list.
    if len(json_error_catcher) > 1: json_totalday['errors'].append(json_row['error'])
    
    
    json_totalseason['period_data'].append(json_totalday)
    print(single_day.strftime('%m-%d'),'done')

runtime = time.time() - program_start_time
json_totalseason['runtime']=runtime
print('COMPLETE')     
print("--- %s seconds ---" % (runtime)

'''
import json
with open('data.json', 'w') as outfile:
    json.dump(json_totalseason, outfile)
'''

In [ ]:

json_totalseason['period_data'][0]['errors']

for day in json_totalseason['period_data']:
    team_count = {}
    print(day['date'],day['day_count'])
    for row in day['data']:
        team_count[row['subreddit']] = team_count.get(row['subreddit'],0) + 1
    
    print(sum(team_count.values()))





In [4]:

'''
subreddits_list = list of 2 item lists that contains shorthand city name /r/<subname>. ['LAL','lakers']

date_range = tuple of date objects

def reddit_Submissions_toJSON(subreddits_list,date_range):
    ([list of ['LAL','lakers']],(start_dateobject,end_dateobject)) -> returns JSON data for all submissions in this period
    
    
reddit_Submissions_toJSON(subreddits_list,season_dates['2014]['playoffs])


'''
def reddit_Submissions_toJSON(subreddits_list,date_range):
    '''
    (list,tuple) -> 
    
    '''
    
    #load oauth credentials and create reddit instance
    from reddit_config import client_id,password,user_agent,username
    reddit = praw.Reddit(client_id=client_id,
                            client_secret=password,
                            user_agent=user_agent,
                            username=username)

    
    #start day =  first day of playoffs
    #end date = last day of playoffs
    start_date = to_date_object(date_range[0])
    end_date = to_date_object(date_range[1])

    
    program_start_time = time.time()


    json_totalseason = {'data_generated_date':datetime.now().strftime("%a, %d %b %Y %H:%M:%S -0800"),
                        'period_data':[],'runtime':''}
    #for each day during the playoffs
    for single_day in daterange(start_date,end_date):
        day_start = time.mktime(datetime.combine(single_day,time_(0,0,0)).timetuple())
        day_end = time.mktime(datetime.combine(single_day,time_(23,59,59)).timetuple())

        #this dicts holds data for the total day. Each day is stored as a list of dicts under 'data' key
        #date is date for the day of data, day_count is total # of submissions across all nba subreddits
        #errors logs any days for any subreddits where the data extraction process failed
        json_totalday = {'date':'','day_count':0,'data':[],'errors':[]}
        #counter for total submissions per day
        total_subs = 0
        for team in subreddits_list:
            #initializes our error catcher for the day
            json_error_catcher = []
            #get the /r/subredditname and pass it to praw reddit object
            subreddit = reddit.subreddit(team[1])

            #if this try fails that means data extraction failed for a particular day, so log the day and subreddit
            try:
                #for all submissions for 1 subreddit from start of day to end
                for submission in subreddit.submissions(day_start,day_end):
                    #json_row holds our row data
                    json_row = {'title':None,
                                'subreddit':None,
                                'author':None,
                                'create_date':None,
                                'submission_id':None,
                                'num_comments':None}

                    #increment our total sub counter
                    total_subs+=1

                    #get all the data values wanted
                    json_row['title']=submission.title
                    json_row['subreddit']=team[0]
                    json_row['author'] = submission.author.name
                    json_row['create_date'] = submission.created_utc
                    json_row['submission_id'] = submission.id
                    json_row['num_comments'] = submission.num_comments

                    #appends the json data row to list in totalday data. If extraction failed the values will be None    
                    json_totalday['data'].append(json_row)


            except:
                #if extraction fails, create a list of 2 items, the string date and string name for subreddit
                #and append it to json_error_catcher list
                json_error = {'error':True,'team':team[0],'day':single_day.strftime('%Y-%m-%d')}
                json_totalday['data'].append(json_error)
                json_error_catcher.append([single_day.strftime('%Y-%m-%d'),team[0]])



        #records the date
        json_totalday['date'] = single_day.strftime('%Y-%m-%d')
        #records total subs for day
        json_totalday['day_count'] = total_subs

        #if there was an error then append the errors to the 'errors' list in json_totalday for later inspection
        #if no errors then json_totalday['errors] will be an empty list.
        if len(json_error_catcher) > 1: json_totalday['errors'].append(json_row['error'])


        json_totalseason['period_data'].append(json_totalday)
        #print(single_day.strftime('%m-%d'),'done')

    runtime = time.time() - program_start_time
    json_totalseason['runtime']=runtime
    print('COMPLETE')     
    print("--- %s seconds ---" % (runtime))
          
    return json_totalseason


Runs the function and creates json files for each NBA season

In [5]:
for year in season_dates:
    for season_type in season_dates[year]:
        test_dict = {}
        test_dict = reddit_Submissions_toJSON(subreddits_list,season_dates[year][season_type])
        test_dict['season_type']=season_type
        test_dict['nba year'] = year
        
        with open('{}-{}.json'.format(year,season_type), 'w') as outfile:
            json.dump(test_dict, outfile)

Version 5.1.0 of praw is outdated. Version 5.2.0 was released Tuesday October 24, 2017.
COMPLETE
--- 1404.31862282753 seconds ---
COMPLETE
--- 1462.404468536377 seconds ---
COMPLETE
--- 584.7969391345978 seconds ---
COMPLETE
--- 1758.0287728309631 seconds ---
COMPLETE
--- 2044.4512968063354 seconds ---
COMPLETE
--- 682.819149017334 seconds ---
COMPLETE
--- 1501.9678287506104 seconds ---
COMPLETE
--- 2117.67236828804 seconds ---
COMPLETE
--- 796.2280328273773 seconds ---
COMPLETE
--- 1550.2451255321503 seconds ---
COMPLETE
--- 2323.0882914066315 seconds ---
COMPLETE
--- 787.9180357456207 seconds ---


In [27]:
import json
big_stitcher = []
counter = 0

for year in season_dates:
    for season_type in season_dates[year]:
        data = json.load(open('{}-{}.json'.format(year,season_type)))
        for day in data['period_data']:
            for row in day['data']:
                big_stitcher.append(row)


In [28]:
len(big_stitcher)

556484

In [29]:
with open('json_dump.json','w') as out:
    json.dump(big_stitcher,out)

In [32]:
from pymongo import MongoClient
client=MongoClient()

In [34]:
db = client.nba_reddit

In [35]:
db['main']

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'nba_reddit'), 'main')

In [37]:
db.main.insert_many(big_stitcher)

In [52]:
db.main.find_one({'subreddit':'LAL'})


{'_id': ObjectId('5a15d2613a162e17e49f5b61'),
 'author': 'schnide1',
 'create_date': 1371884292.0,
 'num_comments': 16,
 'submission_id': '1gug4i',
 'subreddit': 'LAL',
 'title': "Even Brazilian Laker fans want D'Antoni out..."}